In [31]:
import librosa 
import numpy as np
def extract_features(file_name):
   
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccsscaled = np.mean(mfccs.T,axis=0)
        
    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None 
     
    return mfccsscaled

In [32]:
# Load various imports 
import pandas as pd
import os
import librosa

# Set the path to the full UrbanSound dataset 
fulldatasetpath = '../dataset/audio/'

metadata = pd.read_csv('../dataset/dataset.csv')
#metadata = pd.read_csv('../dataset/datasetM2.csv')

features = []

# Iterate through each sound file and extract the features 
for index, row in metadata.iterrows():
    
    file_name = os.path.join(os.path.abspath(fulldatasetpath),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
    
    class_label = row["class_name"]
    data = extract_features(file_name)
    
    features.append([data, class_label])

# Convert into a Panda dataframe 
featuresdf = pd.DataFrame(features, columns=['feature','class_label'])

print('Finished feature extraction from ', len(featuresdf), ' files') 

Finished feature extraction from  1235  files


In [33]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Convert features and corresponding classification labels into numpy arrays
X = np.array(featuresdf.feature.tolist())
y = np.array(featuresdf.class_label.tolist())

# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y)) 

In [34]:
# split the dataset 
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 42)

In [35]:
### store the preprocessed data for use in the next notebook

%store x_train 
%store x_test 
%store y_train 
%store y_test 
%store yy 
%store le

Stored 'x_train' (ndarray)
Stored 'x_test' (ndarray)
Stored 'y_train' (ndarray)
Stored 'y_test' (ndarray)
Stored 'yy' (ndarray)
Stored 'le' (LabelEncoder)


In [36]:
# retrieve the preprocessed data from previous notebook

%store -r x_train 
%store -r x_test 
%store -r y_train 
%store -r y_test 
%store -r yy 
%store -r le

In [37]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 

num_labels = yy.shape[1]
filter_size = 2

# Construct model 
model = Sequential()

model.add(Dense(256, input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [38]:
# Compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam') 

In [39]:
# Display model architecture summary 
model.summary()

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=0)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 256)               10496     
_________________________________________________________________
activation_4 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               65792     
_________________________________________________________________
activation_5 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 4)                

In [40]:
from keras.callbacks import ModelCheckpoint 
from datetime import datetime 

num_epochs = 100
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.basic_mlp.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Train on 988 samples, validate on 247 samples
Epoch 1/100
988/988 [==============================] - 0s 198us/step - loss: 23.5800 - accuracy: 0.5152 - val_loss: 3.7522 - val_accuracy: 0.7611

Epoch 00001: val_loss improved from inf to 3.75219, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 2/100
988/988 [==============================] - 0s 77us/step - loss: 12.5236 - accuracy: 0.6265 - val_loss: 5.1821 - val_accuracy: 0.6883

Epoch 00002: val_loss did not improve from 3.75219
Epoch 3/100
988/988 [==============================] - 0s 87us/step - loss: 7.3242 - accuracy: 0.6559 - val_loss: 1.7122 - val_accuracy: 0.7045

Epoch 00003: val_loss improved from 3.75219 to 1.71220, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 4/100
988/988 [==============================] - 0s 79us/step - loss: 4.4422 - accuracy: 0.6832 - val_loss: 0.9378 - val_accuracy: 0.6397

Epoch 00004: val_loss improved from 1.71220 to 0.93779, saving model to saved_models/weights.best.

988/988 [==============================] - 0s 77us/step - loss: 0.3064 - accuracy: 0.8634 - val_loss: 0.2055 - val_accuracy: 0.9514

Epoch 00036: val_loss did not improve from 0.20096
Epoch 37/100
988/988 [==============================] - 0s 78us/step - loss: 0.2619 - accuracy: 0.8937 - val_loss: 0.1785 - val_accuracy: 0.9433

Epoch 00037: val_loss improved from 0.20096 to 0.17852, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 38/100
988/988 [==============================] - 0s 77us/step - loss: 0.2539 - accuracy: 0.8836 - val_loss: 0.1671 - val_accuracy: 0.9595

Epoch 00038: val_loss improved from 0.17852 to 0.16711, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 39/100
988/988 [==============================] - 0s 74us/step - loss: 0.2436 - accuracy: 0.8998 - val_loss: 0.1503 - val_accuracy: 0.9636

Epoch 00039: val_loss improved from 0.16711 to 0.15026, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 40/100
988/988 [=================

988/988 [==============================] - 0s 79us/step - loss: 0.0653 - accuracy: 0.9757 - val_loss: 0.0394 - val_accuracy: 0.9919

Epoch 00072: val_loss did not improve from 0.03389
Epoch 73/100
988/988 [==============================] - 0s 78us/step - loss: 0.0712 - accuracy: 0.9727 - val_loss: 0.0314 - val_accuracy: 0.9798

Epoch 00073: val_loss improved from 0.03389 to 0.03136, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 74/100
988/988 [==============================] - 0s 79us/step - loss: 0.0615 - accuracy: 0.9767 - val_loss: 0.0318 - val_accuracy: 0.9960

Epoch 00074: val_loss did not improve from 0.03136
Epoch 75/100
988/988 [==============================] - 0s 77us/step - loss: 0.0811 - accuracy: 0.9666 - val_loss: 0.0447 - val_accuracy: 0.9879

Epoch 00075: val_loss did not improve from 0.03136
Epoch 76/100
988/988 [==============================] - 0s 79us/step - loss: 0.0626 - accuracy: 0.9798 - val_loss: 0.0281 - val_accuracy: 0.9960

Epoch 00076: val_

In [41]:
# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training Accuracy:  0.9949392676353455
Testing Accuracy:  0.9959514141082764


In [42]:
import librosa 
import numpy as np 

def extract_feature(file_name):
   
    try:
        audio_data, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=40)
        mfccsscaled = np.mean(mfccs.T,axis=0)
        
    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None, None

    return np.array([mfccsscaled])


In [43]:
def print_prediction(file_name):
    prediction_feature = extract_feature(file_name) 

    predicted_vector = model.predict_classes(prediction_feature)
    predicted_class = le.inverse_transform(predicted_vector) 
    print("The predicted class is:", predicted_class[0], '\n') 

    predicted_proba_vector = model.predict_proba(prediction_feature) 
    predicted_proba = predicted_proba_vector[0]
    for i in range(len(predicted_proba)): 
        category = le.inverse_transform(np.array([i]))
        print(category[0], "\t\t : ", format(predicted_proba[i], '.32f') )

In [49]:
# test

filename = '../dataset/audio/instatest.wav' 
print_prediction(filename) 

The predicted class is: social media 

Education 		 :  0.46335220336914062500000000000000
Shopping 		 :  0.00276486482471227645874023437500
game 		 :  0.00000012299682339289574883878231
social media 		 :  0.53388279676437377929687500000000
